<a href="https://colab.research.google.com/github/velosomn/M5-methods/blob/master/Criptotroxa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os, sys, gc, time, warnings, pickle, psutil, random
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from math import ceil

warnings.filterwarnings('ignore')
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

In [3]:
## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [4]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
## Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition"

Mounted at /content/gdrive
 asset_details.csv		 example_test.csv
'Cópia de criptotroxa.ipynb'	 supplemental_train.csv.zip
 example_sample_submission.csv	 treinocompactado.zip


In [7]:
import zipfile
base_treino=0
with zipfile.ZipFile("gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/treinocompactado.zip", 'r') as zip_ref:
  base_treino = pd.read_csv(zip_ref.open('train.csv'))

In [8]:
base_treino["dia_semana"]=pd.to_datetime(base_treino['timestamp'], unit='s').dt.dayofweek
base_treino['hora'] = pd.to_datetime(base_treino['timestamp'], unit='s').dt.hour

In [9]:
base_treino.sample(100)

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora
11316860,1574431980,12,205.0,0.053470,0.054400,0.053308,0.053720,919324.957443,0.053559,-0.000616,4,14
8497254,1560764460,4,5.0,0.003166,0.003168,0.003165,0.003165,13.000000,0.003166,0.000948,0,9
14515921,1589821740,13,64.0,0.015339,0.015350,0.015302,0.015321,646149.324300,0.015327,-0.000809,0,17
2197546,1529003460,3,97.0,0.169999,0.170700,0.169211,0.169767,575864.315318,0.169583,0.003310,3,19
18150868,1606082940,2,105.0,295.970000,296.500000,295.440000,295.920000,140.051132,295.989065,-0.004833,6,22
...,...,...,...,...,...,...,...,...,...,...,...,...
8274887,1559734800,7,64.0,8.119333,8.137500,8.097700,8.112567,2225.042293,8.115380,0.003409,2,11
12480384,1580095080,2,123.0,357.858000,358.710000,357.270000,358.182000,142.413375,358.000379,-0.001328,0,3
19351520,1611240600,1,6227.0,31594.775714,31631.000000,31355.000000,31506.675714,518.227601,31518.853738,-0.001976,3,14
1730224,1526481780,2,21.0,1258.958000,1263.270000,1255.320000,1258.674000,15.808233,1257.651462,0.001893,2,14


In [10]:
base_treino.shape

(24236806, 12)

In [11]:
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(base_treino.memory_usage(index=True).sum())))

     Reduced grid_df:   2.2GiB


# 1. Estruturar uma base por asset

In [12]:
assets = {x: base_treino[base_treino.Asset_ID == x].drop('Asset_ID', axis=1) for x in base_treino.Asset_ID.unique()}

In [13]:
assets[0]["Target"].head()

1    -0.014399
9    -0.015875
17   -0.015410
25   -0.012524
33   -0.005940
Name: Target, dtype: float64

In [14]:
#base_treino = reduce_mem_usage(base_treino)

Exemplo :
    assets = {2:base_treino[base_treino.Asset_ID==2],3:base_treino[base_treino.Asset_ID==3]...}

In [15]:
assets[0].head()

,timestamp,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora
1,1514764860,5.0,8.5300,8.5300,8.5300,8.5300,78.38,8.530000,-0.014399,0,0
9,1514764920,7.0,8.5300,8.5300,8.5145,8.5145,71.39,8.520215,-0.015875,0,0
17,1514764980,45.0,8.5065,8.5299,8.4848,8.4848,1546.82,8.501394,-0.015410,0,0
25,1514765040,14.0,8.5009,8.5066,8.4744,8.5009,125.80,8.479810,-0.012524,0,0
33,1514765100,5.0,8.5007,8.5007,8.4560,8.4560,125.01,8.458435,-0.005940,0,0


# 2. Tratamento inicial

## 2.1 Timestamp como index

In [16]:
for asset in assets:
    assets[asset] = assets[asset].set_index('timestamp').sort_index()

In [17]:
assets[0].head()

,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora
timestamp,,,,,,,,,,
1514764860,5.0,8.5300,8.5300,8.5300,8.5300,78.38,8.530000,-0.014399,0,0
1514764920,7.0,8.5300,8.5300,8.5145,8.5145,71.39,8.520215,-0.015875,0,0
1514764980,45.0,8.5065,8.5299,8.4848,8.4848,1546.82,8.501394,-0.015410,0,0
1514765040,14.0,8.5009,8.5066,8.4744,8.5009,125.80,8.479810,-0.012524,0,0
1514765100,5.0,8.5007,8.5007,8.4560,8.4560,125.01,8.458435,-0.005940,0,0


## 2.2 Tratamento dos missings

In [18]:
# Função para uso futuro
def preenche_missing():
    return 0

In [19]:
for asset in assets:
    assets[asset] = assets[asset].reindex(range(assets[asset].index[0], assets[asset].index[-1] + 60, 60), method='pad').replace([np.inf, -np.inf], np.nan)

In [20]:
assets[0].info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1956960 entries, 1514764860 to 1632182400
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Count       1956960 non-null  float64
 1   Open        1956960 non-null  float64
 2   High        1956960 non-null  float64
 3   Low         1956960 non-null  float64
 4   Close       1956960 non-null  float64
 5   Volume      1956960 non-null  float64
 6   VWAP        1956960 non-null  float64
 7   Target      1929248 non-null  float64
 8   dia_semana  1956960 non-null  int64  
 9   hora        1956960 non-null  int64  
dtypes: float64(8), int64(2)
memory usage: 149.3 MB


In [21]:
for asset in assets:
    print(f'-----{asset}-----')
    print((assets[asset].index[1:] - assets[asset].index[:-1]).value_counts().head())

-----2-----
60    1956959
Name: timestamp, dtype: int64
-----0-----
60    1956959
Name: timestamp, dtype: int64
-----1-----
60    1956959
Name: timestamp, dtype: int64
-----5-----
60    1956959
Name: timestamp, dtype: int64
-----7-----
60    1956959
Name: timestamp, dtype: int64
-----6-----
60    1956959
Name: timestamp, dtype: int64
-----9-----
60    1956959
Name: timestamp, dtype: int64
-----11-----
60    1956959
Name: timestamp, dtype: int64
-----13-----
60    1903823
Name: timestamp, dtype: int64
-----12-----
60    1889287
Name: timestamp, dtype: int64
-----3-----
60    1803769
Name: timestamp, dtype: int64
-----8-----
60    1772153
Name: timestamp, dtype: int64
-----10-----
60    1770279
Name: timestamp, dtype: int64
-----4-----
60    1285046
Name: timestamp, dtype: int64


# 3. Criação de features

## 3.1 Features gerais - vai ter para todos assets

Fazer a construção de features usando funções, pra facilitar a aplicação nos assets

Features já existentes
* timestamp: All timestamps are returned as second Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data.
* Asset_ID: The asset ID corresponding to one of the crytocurrencies (e.g. Asset_ID = 1 for Bitcoin). The mapping from Asset_ID to crypto asset is contained in asset_details.csv.
* Count: Total number of trades in the time interval (last minute).
* Open: Opening price of the time interval (in USD).
* High: Highest price reached during time interval (in USD).
* Low: Lowest price reached during time interval (in USD).
* Close: Closing price of the time interval (in USD).
* Volume: Quantity of asset bought or sold, displayed in base currency USD.
* VWAP: The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data.
* Target: Residual log-returns for the asset over a 15 minute horizon.

Ideias de features
* Dia da semana
* Feriados Americanos e Chineses
* Volatilidade exponencial - http://ferramentasdoinvestidor.com.br/gestao-de-risco/volatilidade-ewma-no-excel/
* Indice de força relativa
* Médias moveis
* Bandas de bollinger
* MACD
* Agulhada do DIDI - https://www.investimentonabolsa.com/2014/11/saiba-utilizar-agulhada-do-didi.html
* Analise de fourrier - https://journals.sagepub.com/doi/pdf/10.5772/56839
* Nuvens de ichimoku - https://www.investopedia.com/terms/i/ichimoku-cloud.asp
* numero de transações de bitcoin - observar interesse do publico em cripto

In [22]:
#def media_movel_30(precos):
#    return precos.rolling(30).mean()

def media_movel(precos,intervalo):
    return precos.rolling(intervalo).mean().astype("float64")

In [23]:
def get_bollinger_bands(dfbol, rate=30):
    sma = np.array(dfbol.rolling(rate).mean())
    std = np.array(dfbol.rolling(rate).std())
    boll_up =  sma + std * 2 # Calculate top band
    boll_down = sma - std * 2 # Calculate bottom band
    boll=np.zeros(dfbol.size)
    preco=np.array(dfbol)
    indexes = list(range(dfbol.size))
    for i in indexes:
        if boll_up[i]<preco[i]:
            boll[i]=1
        elif boll_down[i]>preco[i]:
            boll[i]=-1
        else:
            boll[i]=0
    return boll

In [24]:
def agulhada_didi(dfagu):
    didi3 = np.array(dfagu.rolling(3).mean())
    didi8 = np.array(dfagu.rolling(8).mean())
    didi20 = np.array(dfagu.rolling(20).mean())
    Didi=np.zeros(dfagu.size)
    indexes = list(range(dfagu.size))
    for i in indexes:
        if ((didi3[i]>didi8[i]) & (didi8[i]>didi20[i])):
            Didi[i]=1
        else:
            Didi[i]=0
    return Didi

In [25]:
    def MACD(dfmacd):
        exp1 = dfmacd.ewm(span=12, adjust=False).mean()
        exp2 = dfmacd.ewm(span=26, adjust=False).mean()
        diff = exp1-exp2
        diff_np=np.array(diff)
        exp3 = np.array(diff.ewm(span=9, adjust=False).mean())
        MACD=np.zeros(dfmacd.size)
        indexes = list(range(dfmacd.size))
        for i in indexes:
            if (diff_np[i]>=exp3[i]):
                MACD[i]=1
            else:
                MACD[i]=-1        
        return MACD

In [26]:
def willR (dfw,dfhigh,dflow,bins):
    high = [np.array(dfhigh.rolling(bins).max(),dtype=float)-np.array(dfw,dtype=float)]
    low = [np.array(dfhigh.rolling(bins).max(),dtype=float)-np.array(dflow.rolling(bins).min(),dtype=float)]
    #
    willr = np.transpose(np.nan_to_num(np.true_divide(high,low,out=np.zeros_like(high), where=low!=0)*(-100)))
    
    willr_flag=np.zeros(dfw.size)
    indexes = list(range(dfw.size))
    for i in indexes:
        if (willr[i]>-20):
            willr_flag[i]=1
        elif (willr[i]<-80):
            willr_flag[i]=-1
        else:
            willr_flag[i]=0

    return willr,willr_flag

In [27]:
def log_return(series, periods=1):
    return np.log(series.astype("float64")).diff(periods=periods)

In [28]:
def upper_shadow_percent(df): return (df['High'] / np.maximum(df['Close'], df['Open'])) -1
def lower_shadow_percent(df): return (np.minimum(df['Close'], df['Open']) / df['Low']) -1

def upper_shadow_15_perc(df): return (df['High'].rolling(window=15, min_periods=1).max() / np.maximum(df['Close'], df['Open']).shift(15)) -1
def lower_shadow_15_perc(df): return (np.minimum(df['Close'], df['Open']).shift(15) / df['Low'].rolling(window=15, min_periods=1).min()) -1

In [29]:
for base in assets:
    
    #simple moving average
    for i in [7,15,30,60,180]:
      assets[base]['SMA_'+str(i)] = media_movel(assets[base]['Close'],i)
      assets[base]['SMA_'+str(i)].fillna(method='bfill',inplace=True)
    
    #log simple moving average
    for i in [7,14,30,60,180]:
      assets[base]['log_SMA_'+str(i)] = np.log(media_movel(assets[base]['Close'],i))
      assets[base]['log_SMA_'+str(i)].fillna(method='bfill',inplace=True)

    
    #log cumulative moving average
    assets[base]['log_CMA_30'] = np.log(assets[base]['Close'].expanding(30).mean())
    assets[base]['log_CMA_15'] = np.log(assets[base]['Close'].expanding(15).mean())
    assets[base]['log_CMA_30'].fillna(method='bfill',inplace=True)
    assets[base]['log_CMA_15'].fillna(method='bfill',inplace=True)

    
    #log exponential moving average
    assets[base]['log_EWM_30'] = np.log(assets[base]['Close'].ewm(span=30).mean())
    assets[base]['log_EWM_15'] = np.log(assets[base]['Close'].ewm(span=15).mean())
    assets[base]['log_EWM_30'].fillna(method='bfill',inplace=True)
    assets[base]['log_EWM_15'].fillna(method='bfill',inplace=True)



    #Bollinger
    assets[base]['boll_30'] = get_bollinger_bands(assets[base]['Close'],30)
    #DIDI Index
    assets[base]['Didi'] = agulhada_didi(assets[base]['Close'])
    #MACD
    assets[base]['MACD'] = MACD(assets[base]['Close'])
    #Williams R
    assets[base]['willr'],assets[base]['willr_flag'] = willR(assets[base]['Close'],assets[base]['High'],assets[base]['Low'],14)
    #Log do Retorno
    assets[base]["log_ret"]=log_return(assets[base]['Close'])[1:]
    assets[base]['log_ret'].fillna(method='bfill',inplace=True)

    assets[base]["upper_shadow_perc"]=upper_shadow_percent(assets[base])[1:]
    assets[base]['upper_shadow_perc'].fillna(method='bfill',inplace=True)
    assets[base]["lower_shadow_perc"]=lower_shadow_percent(assets[base])[1:]
    assets[base]['lower_shadow_perc'].fillna(method='bfill',inplace=True)

    assets[base]["upper_shadow_15_perc"]=upper_shadow_15_perc(assets[base])[1:]
    assets[base]['upper_shadow_15_perc'].fillna(method='bfill',inplace=True)

    assets[base]["lower_shadow_15_perc"]=lower_shadow_15_perc(assets[base])[1:]
    assets[base]['lower_shadow_15_perc'].fillna(method='bfill',inplace=True)

    assets[base]["Target"].fillna(method='pad',inplace=True)



In [30]:
for base in assets:
  assets[base]=reduce_mem_usage(assets[base])

Mem. usage decreased to 123.18 Mb (75.7% reduction)
Mem. usage decreased to 126.91 Mb (75.0% reduction)
Mem. usage decreased to 126.91 Mb (75.0% reduction)
Mem. usage decreased to 130.64 Mb (74.3% reduction)
Mem. usage decreased to 130.64 Mb (74.3% reduction)
Mem. usage decreased to 130.64 Mb (74.3% reduction)
Mem. usage decreased to 126.91 Mb (75.0% reduction)
Mem. usage decreased to 123.18 Mb (75.7% reduction)
Mem. usage decreased to 123.46 Mb (75.0% reduction)
Mem. usage decreased to 122.52 Mb (75.0% reduction)
Mem. usage decreased to 120.41 Mb (74.3% reduction)
Mem. usage decreased to 114.92 Mb (75.0% reduction)
Mem. usage decreased to 111.43 Mb (75.7% reduction)
Mem. usage decreased to 85.79 Mb (74.3% reduction)


In [31]:
del base_treino

In [ ]:
pk.dump(assets,open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/base_mod.pkl','wb'))

In [32]:
assets[0].sample(100)

,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora,SMA_7,SMA_15,SMA_30,SMA_60,SMA_180,log_SMA_7,log_SMA_14,log_SMA_30,log_SMA_60,log_SMA_180,log_CMA_30,log_CMA_15,log_EWM_30,log_EWM_15,boll_30,Didi,MACD,willr,willr_flag,log_ret,upper_shadow_perc,lower_shadow_perc,upper_shadow_15_perc,lower_shadow_15_perc
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1611161820,272.0,41.968750,42.000000,41.906250,42.000000,4713.448242,41.968750,-0.001318,2,16,42.031250,42.000000,41.875000,41.406250,41.406250,3.738281,3.738281,3.734375,3.722656,3.722656,2.884766,2.884766,3.734375,3.736328,0.0,0.0,-1.0,-49.718750,0.0,-0.000714,0.000600,0.001637,0.004936,0.004776
1612950000,2374.0,143.500000,143.625000,142.500000,142.750000,13368.830078,143.000000,-0.006847,2,9,144.000000,144.625000,145.500000,145.625000,140.000000,4.968750,4.972656,4.980469,4.980469,4.941406,2.917969,2.917969,4.976562,4.972656,-1.0,0.0,-1.0,-95.750000,-1.0,-0.005753,0.001230,0.001117,-0.002850,0.025543
1593507720,13.0,15.382812,15.390625,15.382812,15.382812,282.950012,15.382812,0.002716,1,9,15.375000,15.382812,15.382812,15.375000,15.390625,2.732422,2.732422,2.732422,2.732422,2.734375,2.757812,2.757812,2.732422,2.732422,0.0,0.0,1.0,-20.296875,0.0,0.000026,0.000071,0.000032,0.000227,0.001360
1574024100,28.0,20.078125,20.093750,20.062500,20.093750,440.040009,20.078125,-0.000838,6,20,20.093750,20.125000,20.171875,20.171875,20.187500,3.000000,3.001953,3.003906,3.003906,3.003906,2.746094,2.746094,3.001953,3.001953,0.0,0.0,-1.0,-83.125000,-1.0,0.000503,0.000204,0.000633,0.000648,0.006855
1606353420,178.0,31.437500,31.500000,31.437500,31.468750,4273.525879,31.468750,0.001023,3,1,31.531250,31.609375,31.593750,31.593750,31.484375,3.451172,3.453125,3.453125,3.453125,3.449219,2.833984,2.833984,3.453125,3.451172,0.0,0.0,-1.0,-85.687500,-1.0,0.000855,0.001093,0.000309,0.007584,0.002417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542381120,16.0,8.101562,8.101562,8.085938,8.085938,2540.989990,8.085938,-0.002024,4,15,8.101562,8.117188,8.140625,8.156250,8.156250,2.091797,2.093750,2.097656,2.099609,2.099609,2.478516,2.478516,2.095703,2.093750,0.0,0.0,-1.0,-92.000000,-1.0,-0.002348,0.000000,0.000433,0.000000,0.009171
1596454680,190.0,22.406250,22.437500,22.375000,22.406250,3874.399902,22.406250,-0.002296,0,11,22.390625,22.390625,22.250000,22.125000,21.859375,3.109375,3.109375,3.103516,3.095703,3.083984,2.763672,2.763672,3.103516,3.107422,0.0,0.0,-1.0,-37.250000,0.0,0.001500,0.000767,0.001466,0.009621,-0.002895
1518525060,20.0,8.945312,8.953125,8.937500,8.945312,801.349976,8.945312,0.005596,1,12,8.937500,8.921875,8.906250,8.898438,8.851562,2.189453,2.189453,2.185547,2.185547,2.181641,2.566406,2.566406,2.187500,2.189453,0.0,1.0,1.0,-1.877930,1.0,-0.000022,0.000201,0.001007,0.003836,0.008118


In [33]:
assets[1].isnull().sum()

Count                   0
Open                    0
High                    0
Low                     0
Close                   0
Volume                  0
VWAP                    0
Target                  0
dia_semana              0
hora                    0
SMA_7                   0
SMA_15                  0
SMA_30                  0
SMA_60                  0
SMA_180                 0
log_SMA_7               0
log_SMA_14              0
log_SMA_30              0
log_SMA_60              0
log_SMA_180             0
log_CMA_30              0
log_CMA_15              0
log_EWM_30              0
log_EWM_15              0
boll_30                 0
Didi                    0
MACD                    0
willr                   0
willr_flag              0
log_ret                 0
upper_shadow_perc       0
lower_shadow_perc       0
upper_shadow_15_perc    0
lower_shadow_15_perc    0
dtype: int64

## 3.2 Features específicas (opcionais)

### 3.2.1 BTC

### 3.2.1 ETH

# 4. Modelagem

## 4.1 Quebra de treino/validação/teste

In [ ]:
# list_periodos = []

# for asset in base_treino.Asset_ID.unique():
#     maximo = base_treino[base_treino.Asset_ID == asset].timestamp.max()
#     minimo = base_treino[base_treino.Asset_ID == asset].timestamp.min()
#     list_periodos.append([asset,minimo,maximo])
    
# df_periodos = pd.DataFrame(list_periodos, columns=['asset','min','max'])
# df_periodos['min'] = pd.to_datetime(df_periodos['min'], unit='s')
# df_periodos['max'] = pd.to_datetime(df_periodos['max'], unit='s')
# df_periodos.sort_values('min')

In [34]:
y_treino={x: np.array(assets[x].loc[totimestamp("01/01/2020"):totimestamp("01/04/2021")]["Target"]) for x in assets.keys()}

In [35]:
y_teste={x: np.array(assets[x].loc[totimestamp("01/04/2021"):totimestamp("01/07/2021")]["Target"]) for x in assets.keys()}

In [36]:
x_treino = {x: assets[x].loc[totimestamp("01/01/2020"):totimestamp("01/04/2021")].drop(['Open', 'High', 'Low', 'Close', 'VWAP','Target'], axis=1) for x in assets.keys()}

In [88]:
x_teste = {x: assets[x].loc[totimestamp("01/04/2021"):totimestamp("01/07/2021")].drop(['Open', 'High', 'Low', 'Close', 'VWAP','Target'], axis=1) for x in assets.keys()}

## 4.2 Modelo baseline

## 4.3 Modelagem freestyle

4.4 MODELAGEM MARCELO

In [ ]:
params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 5,
    "learning_rate": 0.2,
    "loss": "squared_error",
}

In [ ]:
predict_treino_XGB={}
predict_teste_XGB ={}
models_XGB ={}
for base in assets:
  reg = xgb.XGBRegressor(objective="reg:squarederror",**params)
  models_XGB[base]=reg.fit(x_treino[base], y_treino[base])
  predict_treino_XGB=reg.predict(x_treino[base])
  predict_teste_XGB=reg.predict(x_teste[base])

In [ ]:
predito[0]

array([ 8.5085630e-05, -2.3663044e-05, -2.2113323e-05, ...,
       -4.6122074e-04, -4.3457747e-04,  2.1457672e-05], dtype=float32)

In [ ]:
dict_importances=reg._Booster.get_score(importance_type="weight")
# names_vars=pd.DataFrame(list(dict_importances.keys()),columns=["importancia"])
# importancia=pd.concat
dict_importances

{'Count': 377,
 'Didi': 8,
 'MACD': 17,
 'SMA_15': 238,
 'SMA_180': 385,
 'SMA_30': 240,
 'SMA_60': 261,
 'SMA_7': 356,
 'Volume': 252,
 'boll_30': 18,
 'dia_semana': 117,
 'hora': 271,
 'log_CMA_30': 239,
 'log_EWM_15': 38,
 'log_EWM_30': 36,
 'log_SMA_14': 43,
 'log_SMA_180': 31,
 'log_SMA_30': 25,
 'log_SMA_60': 24,
 'log_SMA_7': 17,
 'log_ret': 154,
 'lower_shadow_15_perc': 288,
 'lower_shadow_perc': 151,
 'upper_shadow_15_perc': 272,
 'upper_shadow_perc': 161,
 'willr': 202}

In [ ]:
for base in assets:
  mse = mean_squared_error(y_teste[base], reg.predict(x_teste[base]))
  print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

NameError: ignored

4.45 Modelo XGB com parameters Sampler 

In [ ]:
#Com parameter sampler

In [47]:
from sklearn.model_selection import ParameterSampler
param_dist = {
    "n_estimators": [100,500],
    "max_depth": [4,6,10],
    "colsample_bytree":[0.9],
    "min_samples_split": [2,4,5],
    "reg_lambda":[1,7],
    "reg_alpha": [0,6],
    "scale_pos_weight": [15],
    "learning_rate": [0.2],
    "gamma":[0.01,0.1,0.5]
}
params_set=ParameterSampler(param_dist,15,random_state=0)

In [48]:
from typing import Tuple
import xgboost as xgb
def rmsle(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    ''' Root mean squared log error metric.'''
    y = dtrain.get_label()
    predt[predt < -1] = -1 + 1e-6
    elements = np.power(np.log1p(y) - np.log1p(predt), 2)
    return 'PyRMSLE', float(np.sqrt(np.sum(elements) / len(y)))

In [49]:
def arruma_n_estimators(row):
  best_iter=row["best_iter"]
  params_inicial = row["params_iter"]
  params_com_early = {**params_inicial, **{'n_estimators':best_iter +1}}
  return params_com_early

In [50]:
from tqdm import tqdm_notebook
import tqdm

In [51]:
# https://xgboost.readthedocs.io/en/stable/tutorials/custom_metric_obj.html

In [52]:
def roda_grid(params,x_train,y_train,eval_set,eval_metric,early_stopping_rounds=7):
  for k,i in tqdm.tqdm(enumerate(params,1)):
    regressor=xgb.XGBRegressor (objective="reg:squarederror", seed=0,**i)

    regressor.fit(x_train,y_train,eval_set=eval_set,eval_metric=eval_metric,verbose=False,early_stopping_rounds=early_stopping_rounds)

    if k==1:
      df_result=pd.DataFrame()
      df_result["best_treino"]=[regressor.evals_result()["validation_0"]['PyRMSLE'][regressor.best_iteration]]
      df_result["best_test"]=[regressor.evals_result()["validation_1"]['PyRMSLE'][regressor.best_iteration]]
      df_result["best_iter"]=regressor.best_iteration
      df_result["Combination"]=k
      df_result["params_iter"]=[i.copy()]
    else:
      aux=pd.DataFrame()
      aux["best_treino"]=[regressor.evals_result()["validation_0"]['PyRMSLE'][regressor.best_iteration]]
      aux["best_test"]=[regressor.evals_result()["validation_1"]['PyRMSLE'][regressor.best_iteration]]
      aux["best_iter"]=regressor.best_iteration
      aux["Combination"]=k
      aux["params_iter"]=[i.copy()]


      df_result=df_result.append(aux).reset_index(drop=True)
      df_result["params_com_early"]=df_result.apply(arruma_n_estimators, axis=1)
      df_result["diff"]=df_result["best_treino"] - df_result["best_test"]
      df_result=(df_result.sort_values(by=["best_test","diff", "best_iter"],ascending=[True,True,True]).reset_index(drop=True))

  return df_result

In [56]:
df_result={}
best_params={}
eval_set={}
for base in assets:
  eval_set[base] = [(x_treino[base],y_treino[base]),(x_teste[base],y_teste[base])]
  eval_metric=rmsle
  print(base)
  df_result[base]=roda_grid(params_set, x_treino[base], y_treino[base],eval_set[base],eval_metric=rmsle)
  best_params[base]=df_result[base].loc[0,"params_com_early"]


2


15it [16:41, 66.75s/it]


0


15it [15:30, 62.05s/it]


1


...

15it [16:28, 65.92s/it]


10


15it [13:36, 54.45s/it]


4


15it [16:14, 64.98s/it]


In [59]:
predict_treino_XGB={}
predict_teste_XGB ={}
modelos ={}
import pickle as pk
for base in assets:
  modelos[base] = xgb.XGBRegressor(objective="reg:squarederror",**best_params[base])
  modelos[base].fit(x_treino[base], y_treino[base],eval_metric=rmsle,eval_set=eval_set[base])
  pk.dump(modelos[base],open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/modelo_'+str(base)+'.pkl','wb'))

[0]	validation_0-rmse:0.400046	validation_1-rmse:0.400028	validation_0-PyRMSLE:0.336516	validation_1-PyRMSLE:0.336519
[1]	validation_0-rmse:0.319867	validation_1-rmse:0.320031	validation_0-PyRMSLE:0.277676	validation_1-PyRMSLE:0.277684
[2]	validation_0-rmse:0.256138	validation_1-rmse:0.256039	validation_0-PyRMSLE:0.227978	validation_1-PyRMSLE:0.227993
[3]	validation_0-rmse:0.204839	validation_1-rmse:0.20485	validation_0-PyRMSLE:0.186363	validation_1-PyRMSLE:0.186386
[4]	validation_0-rmse:0.163909	validation_1-rmse:0.163905	validation_0-PyRMSLE:0.151779	validation_1-PyRMSLE:0.151812
[5]	validation_0-rmse:0.131125	validation_1-rmse:0.131156	validation_0-PyRMSLE:0.123226	validation_1-PyRMSLE:0.123272
[6]	validation_0-rmse:0.104918	validation_1-rmse:0.104967	validation_0-PyRMSLE:0.099786	validation_1-PyRMSLE:0.099847
[7]	validation_0-rmse:0.083957	validation_1-rmse:0.084026	validation_0-PyRMSLE:0.080634	validation_1-PyRMSLE:0.080714
[8]	validation_0-rmse:0.067197	validation_1-rmse:0.067289

In [75]:

for base in assets:
  del modelos[base]

In [76]:
pk.dump(assets,open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/base_mod.pkl','wb'))

In [77]:
# dict_importances=modelos[0]._Booster.get_score(importance_type="weight")
# names_vars= pd.DataFrame(list(dict_importances.keys()),columns=["Vars"])
# importancia_vars = pd.DataFrame(list(dict_importances.values()),columns=["importancia"])
# importancia=pd.concat([names_vars,importancia_vars],axis=1).sort_values(by="importancia")
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.figure(figsize=(10,20))
# plt.barh(importancia["Vars"],importancia["importancia"])

In [87]:
x_valid = {x: assets[x].loc[totimestamp("01/07/2021"):totimestamp("01/09/2021")].drop(['Open', 'High', 'Low', 'Close', 'VWAP','Target'], axis=1) for x in assets.keys()}

In [89]:
y_valid={x: np.array(assets[x].loc[totimestamp("01/07/2021"):totimestamp("01/09/2021")]["Target"]) for x in assets.keys()}

In [90]:
predito_valid={}
for base in assets:
  modelo=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/modelo_'+str(base)+'.pkl','rb'))
  predito_valid[base]=modelo.predict(assets[base].loc[totimestamp("01/07/2021"):totimestamp("01/09/2021")].drop(['Open', 'High', 'Low', 'Close', 'VWAP','Target'], axis=1))


In [91]:
 predito_valid[0]

array([6.318092e-05, 6.318092e-05, 6.318092e-05, ..., 6.318092e-05,
       6.318092e-05, 6.318092e-05], dtype=float32)

In [94]:
from sklearn.metrics import mean_squared_error
[{"Valid":mean_squared_error(predito_valid[0],y_valid[0])}]

[{'Valid': 4.5249963e-06}]

# 5. Avaliação dos resultados